In [1]:
import pandas as pd
import numpy as np
from datetime import date
import re
import warnings

import feature_generation as fg
import imputation as imp
import evaluation as eva
import train_test_split as tts
import clf_define as clfd
import data_util as util

In [2]:
data = util.read_csv_data(r"D:\Project Data\MLHW3\projects_2012_2013.csv", {}, parse_dates=['date_posted', 'datefullyfunded'])

First of all, we will like to define our features and classifiers in the dataframe. In order to maintain the as much patterns as possible, we will only drop the columns with ids which include too much distinguish categories (more than 51) and information like logitude and latitude which can be categorized better using city and state. The number 51 is selected due to the number of state categories.

In [3]:
preserved_columns = []
for column in data.columns:
    if (data[column].dtype=='O') and (len(data[column].unique())<=51):
        preserved_columns.append(column)

In [4]:
print(preserved_columns)

['school_state', 'school_metro', 'school_charter', 'school_magnet', 'teacher_prefix', 'primary_focus_subject', 'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', 'eligible_double_your_impact_match']


In [5]:
continuous_features = ['total_price_including_optional_support', 'students_reached']
categorical_features = set(preserved_columns) - set(continuous_features)

In [6]:
print(categorical_features)

{'teacher_prefix', 'resource_type', 'school_state', 'school_magnet', 'primary_focus_subject', 'secondary_focus_subject', 'school_charter', 'poverty_level', 'grade_level', 'eligible_double_your_impact_match', 'school_metro', 'secondary_focus_area', 'primary_focus_area'}


In [7]:
data[continuous_features + list(categorical_features)].head()

,total_price_including_optional_support,students_reached,teacher_prefix,resource_type,school_state,school_magnet,primary_focus_subject,secondary_focus_subject,school_charter,poverty_level,grade_level,eligible_double_your_impact_match,school_metro,secondary_focus_area,primary_focus_area
0,1498.61,31.0,Mrs.,Supplies,IL,f,Mathematics,Visual Arts,f,highest poverty,Grades PreK-2,f,urban,Music & The Arts,Math & Science
1,282.47,28.0,Mrs.,Books,CA,f,Civics & Government,Literature & Writing,f,highest poverty,Grades 3-5,t,urban,Literacy & Language,History & Civics
2,1012.38,56.0,Ms.,Technology,CA,f,Literacy,Social Sciences,f,high poverty,Grades 3-5,f,urban,History & Civics,Literacy & Language
3,175.33,23.0,Ms.,Books,NY,t,Literacy,NaN,f,high poverty,Grades PreK-2,f,urban,NaN,Literacy & Language
4,3591.11,150.0,Mrs.,Technology,NY,f,Literacy,Literature & Writing,f,high poverty,Grades PreK-2,f,suburban,Literacy & Language,Literacy & Language


We label 1 to the project which are not funded in 60 days right after they are posted, and label 0 to those who are funded in 60 days right after it is posted.

In [8]:
data['duration'] = data["datefullyfunded"] - data["date_posted"]
data['classifier'] =  np.where(data['duration'] > pd.Timedelta('60 days'), 1, 0)

In [9]:
classifier = "classifier"
data["classifier"].head()

0    0
1    0
2    1
3    0
4    1
Name: classifier, dtype: int32

We then move on to split the dataframe to train and test sets with rolling windows stratefy in this case. I split the dataframe with time unit of half years cutoff which is at date 6/30 and 12/31. For every training data set, there will remain 60 days gap for the result (label) of the outcome (classifier) to be observed. The testing set will be set as half year (might be shorter in the last test set) right after the gap.

In [10]:
half_year_stamp = [(1,1), (7,1)]
start_date = pd.Timestamp('2012-01-01 00:00:00')
end_date = pd.Timestamp('2014-01-01 00:00:00')

time_cutoff lists includes the start date, one day after the end dates and other cutoff dates, time interval will be defined as the date between them (including the lower but not include the upper bound).

In [11]:
time_cutoffs = tts.gen_time_cuts(start_date, end_date, half_year_stamp)
print(sorted(time_cutoffs))

[Timestamp('2012-01-01 00:00:00'), Timestamp('2012-07-01 00:00:00'), Timestamp('2013-01-01 00:00:00'), Timestamp('2013-07-01 00:00:00'), Timestamp('2014-01-01 00:00:00')]


I use define_time_variables to create the indicators for rolling window split in this case, taking waiting times (60 days) into account. All train dataframe starts in 2012-01-01 but the testing data starts at 2012-07-01 + (60 days), 2013-01-01 + (60 days), 2013-07-01 + (60 days) respectively.

In [12]:
data = tts.define_time_variables(data, pd.Timedelta('183 days'), pd.Timedelta('60 days'), "date_posted", "datefullyfunded", start_date, end_date, half_year_stamp)

In [13]:
data[data["tmp_label0"] == "train"]["date_posted"].sort_values(ascending=True).head()

69180   2012-01-01
32005   2012-01-01
38220   2012-01-01
74297   2012-01-01
21950   2012-01-01
Name: date_posted, dtype: datetime64[ns]

In [14]:
data[data["tmp_label0"] == "test"]["date_posted"].sort_values(ascending=True).head()

16332    2012-08-30
60107    2012-08-30
71880    2012-08-30
107093   2012-08-30
107156   2012-08-30
Name: date_posted, dtype: datetime64[ns]

In [15]:
data[data["tmp_label1"] == "train"]["date_posted"].sort_values(ascending=True).head()

72487    2012-01-01
63421    2012-01-01
117377   2012-01-01
62941    2012-01-01
117198   2012-01-01
Name: date_posted, dtype: datetime64[ns]

In [16]:
data[data["tmp_label1"] == "test"]["date_posted"].sort_values(ascending=True).head()

121525   2013-03-02
37673    2013-03-02
3158     2013-03-02
108509   2013-03-02
30275    2013-03-02
Name: date_posted, dtype: datetime64[ns]

In [17]:
data[data["tmp_label2"] == "train"]["date_posted"].sort_values(ascending=True).head()

63675    2012-01-01
114946   2012-01-01
48094    2012-01-01
50966    2012-01-01
107459   2012-01-01
Name: date_posted, dtype: datetime64[ns]

In [18]:
data[data["tmp_label2"] == "test"]["date_posted"].sort_values(ascending=True).head()

66491    2013-08-30
121728   2013-08-30
15134    2013-08-30
111442   2013-08-30
93247    2013-08-30
Name: date_posted, dtype: datetime64[ns]

The data is then be splitted according to the time_split_indicators

In [19]:
time_split_indicators = ["tmp_label0", "tmp_label1", "tmp_label2"]

In [20]:
data_dict = tts.rolling_window_split(data, time_split_indicators, list(categorical_features) + continuous_features, classifier)

We than move on to the imputation of missing values in the data of each sub set, we used the trained infromation in the training dataframe is used it in the transformation in the corresponding test dataframe. We will used different strategies with different missing values in the dataframe. For the categorical variables, we will use a "unknown" category to replace the unknown values. (Using fill_unknown) For the continuous variable, we will use the group mean of the training data to impute both training and testing dataframe. (Using fill_na_mean) 

We can observe several missing values in the given dataframe after rolling window split, we do not want to include classifier in the imputation since labeling unknown classification will be problematic with distribution taht we are not sure.

In [21]:
for idx, dat in data_dict.items():
    train_X, train_y, test_X, test_y = dat
    print("missing in training " + idx)
    print(imp.summarize_missing_values(train_X))
    print("missing in testing " + idx)
    print(imp.summarize_missing_values(test_X))
    print("           ")

missing in training tmp_label0
{'teacher_prefix': (0, 26386), 'resource_type': (2, 26386), 'school_state': (0, 26386), 'school_magnet': (0, 26386), 'primary_focus_subject': (2, 26386), 'secondary_focus_subject': (8524, 26386), 'school_charter': (0, 26386), 'poverty_level': (0, 26386), 'grade_level': (1, 26386), 'eligible_double_your_impact_match': (0, 26386), 'school_metro': (2643, 26386), 'secondary_focus_area': (8524, 26386), 'primary_focus_area': (2, 26386), 'students_reached': (4, 26386), 'total_price_including_optional_support': (0, 26386)}
missing in testing tmp_label0
{'teacher_prefix': (0, 33357), 'resource_type': (5, 33357), 'school_state': (0, 33357), 'school_magnet': (0, 33357), 'primary_focus_subject': (4, 33357), 'secondary_focus_subject': (11045, 33357), 'school_charter': (0, 33357), 'poverty_level': (0, 33357), 'grade_level': (2, 33357), 'eligible_double_your_impact_match': (0, 33357), 'school_metro': (4015, 33357), 'secondary_focus_area': (11045, 33357), 'primary_focus_

In [22]:
imp_data_dict = {}
for idx, dat in data_dict.items():
    train_X, train_y, test_X, test_y = dat
    train_X, test_X = imp.fill_na_mean(train_X, test_X, continuous_features)
    train_X, test_X = imp.fill_unknown(train_X, test_X, categorical_features)
    imp_data_dict[idx] = [train_X, train_y, test_X, test_y]

We can observe that after the imputation the new_data_dict contains no missing values in the training and testing subsets

In [23]:
for idx, dat in imp_data_dict.items():
    train_X, train_y, test_X, test_y = dat
    print("missing in training " + idx)
    print(imp.summarize_missing_values(train_X))
    print("missing in testing " + idx)
    print(imp.summarize_missing_values(test_X))
    print("           ")

missing in training tmp_label0
{'teacher_prefix': (0, 26386), 'resource_type': (0, 26386), 'school_state': (0, 26386), 'school_magnet': (0, 26386), 'primary_focus_subject': (0, 26386), 'secondary_focus_subject': (0, 26386), 'school_charter': (0, 26386), 'poverty_level': (0, 26386), 'grade_level': (0, 26386), 'eligible_double_your_impact_match': (0, 26386), 'school_metro': (0, 26386), 'secondary_focus_area': (0, 26386), 'primary_focus_area': (0, 26386), 'students_reached': (0, 26386), 'total_price_including_optional_support': (0, 26386)}
missing in testing tmp_label0
{'teacher_prefix': (0, 33357), 'resource_type': (0, 33357), 'school_state': (0, 33357), 'school_magnet': (0, 33357), 'primary_focus_subject': (0, 33357), 'secondary_focus_subject': (0, 33357), 'school_charter': (0, 33357), 'poverty_level': (0, 33357), 'grade_level': (0, 33357), 'eligible_double_your_impact_match': (0, 33357), 'school_metro': (0, 33357), 'secondary_focus_area': (0, 33357), 'primary_focus_area': (0, 33357), '

After I impute the missing values, I move on to feature generations. In this process, the features generated in the training dataframe have to be consistant to the features in testing dataframe. All categorical variables in the selected features will be transformed to binary features and all continuous variables will be transformed to scaled continuous using MaxMinScale

In [24]:
ft_data_dict = {}
for idx, dat in imp_data_dict.items():
    train_X, train_y, test_X, test_y = dat
    train_X, test_X = fg.min_max_transformation(train_X, test_X, continuous_features)
    ft_data_dict[idx] = [train_X, train_y, test_X, test_y]

In [25]:
ft_data_dict2 = {}
for idx, dat in ft_data_dict.items():
    train_X, train_y, test_X, test_y = dat
    for cat_column in categorical_features:
        train_X, test_X = fg.category_to_binary(train_X, test_X, cat_column)
    ft_data_dict2[idx] = [train_X, train_y, test_X, test_y]

We then define the final dictionary of data using for machine learning pipeline as ft_data_dict2

In [26]:
data_dict = ft_data_dict2

We then begin the training and evaluation process of the data, we will use different methods, different proportion of the populations we might concern, and also conduct cross_validation in the following training and evaluation process. 

Due to the time concern of training models, I will define the models with differnt priorities. Low priority model will only be trained in the simple grid, the medium priority models will be trained in simple and small grid, and high priority models will the trained in all of the grids. The model with their corresponding priorities are presented as the following.
1. Low priority: KNN, Naive Bayes, SVM
2. Medium priority: Bagging, Adaboost, Gradient Boosting, Extra Trees
3. High priority: Decision Tree, Random Forest, Logistics

Note that in the threshold column in performance matrix, the value is represents the percentile of the predicted probability from the given classifiers. I sort the values of the score (or probability) and define the threshold with the given percentile, any score which is higher than the threshold will then be classified as 1. The design is corresponding to the x-axis in precision-recall curve ("percent of population labelled 1 in prediction") when we care about the percent of population with the highest risks.
0. percentile = 100% , percent of population labelled 1 in prediction: 0%
1. percentile = 99% , percent of population labelled 1 in prediction: 1%
2. percentile = 98% , percent of population labelled 1 in prediction: 2%
3. percentile = 95% , percent of population labelled 1 in prediction: 5%
4. percentile = 90% , percent of population labelled 1 in prediction: 10%
5. percentile = 80% , percent of population labelled 1 in prediction: 20%
6. percentile = 70% , percent of population labelled 1 in prediction: 30%
7. percentile = 60% , percent of population labelled 1 in prediction: 40%
8. percentile = 50% , percent of population labelled 1 in prediction: 50%
9. percentile = 0% , percent of population labelled 1 in prediction: 100%

Notice that if percent of population labelled 1 in prediction is 100%, we can estimate the baseline prediction of precision with base strategy to predict 1 for all rows.

The model below provide the full training for all the model list in the grids. The performance matrix contains the information method used, parameters used, evaluation performance including "accuracy", "f1", "recall", "precision", "AUC_ROC", and the corresponding temporal subset passing in. The function can also be specified to plot precision and recall curve the provide the whole pictures of every thresholds (0.99, 0.98, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5) using in classification.

In [ ]:
warnings.filterwarnings("ignore")
cross_val_performance_small = clfd.cross_validate_performance(clfd.clfs, clfd.small, data_dict, [0.99, 0.98, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5], \
                                ["accuracy", "f1", "recall", "precision", "AUC_ROC"])

In [28]:
import pickle
cross_val_performance_small.to_pickle("cross_val_performance_small_final")

In [30]:
ct = cross_val_performance_small

In [48]:
ct[(ct.method=="decision_tree") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                      decision_tree
accuracy                                                         0.713706
f1                                                               0.117856
recall                                                          0.0690523
precision                                                        0.401911
AUC_ROC                                                           0.64889
threshold                                                            0.95
parameters              decision_tree with parameters : {'splitter': '...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [49]:
ct[(ct.method=="random_forest") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                      random_forest
accuracy                                                         0.715585
f1                                                               0.130145
recall                                                          0.0768221
precision                                                        0.425455
AUC_ROC                                                          0.630976
threshold                                                            0.95
parameters              random_forest with parameters : {'n_jobs': 2, ...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [50]:
ct[(ct.method=="logistics") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                          logistics
accuracy                                                         0.714857
f1                                                                0.12792
recall                                                          0.0755089
precision                                                        0.418182
AUC_ROC                                                          0.565274
threshold                                                            0.95
parameters              logistics with parameters : {'fit_intercept': ...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [32]:
ct[(ct.method=="ET") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                                 ET
accuracy                                                         0.707038
f1                                                               0.104004
recall                                                           0.061392
precision                                                            0.34
AUC_ROC                                                          0.563977
threshold                                                            0.95
parameters              ET with parameters : {'n_jobs': -1, 'class_wei...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [47]:
ct[(ct.method=="gradientboost") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                      gradientboost
accuracy                                                         0.723131
f1                                                             0.00283812
recall                                                         0.00142263
precision                                                        0.565217
AUC_ROC                                                           0.52255
threshold                                                            0.95
parameters              gradientboost with parameters : {'n_estimators...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [34]:
ct[(ct.method=="adaboost") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                           adaboost
accuracy                                                         0.713584
f1                                                               0.124027
recall                                                          0.0732108
precision                                                        0.405455
AUC_ROC                                                          0.634503
threshold                                                            0.95
parameters              adaboost with parameters : {'n_estimators': 10...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [35]:
ct[(ct.method=="bagging") & (ct.cross_validate_index=="tmp_label2") & (ct.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                            bagging
accuracy                                                         0.714615
f1                                                               0.127178
recall                                                          0.0750711
precision                                                        0.415758
AUC_ROC                                                          0.566985
threshold                                                            0.95
parameters              bagging with parameters : {'base_estimator__C'...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [39]:
other = {'SVM' :{'C' :[0.001,0.01,0.1,1,10],'kernel':['linear']},
'KNN' :{'n_neighbors': [1,5,10],'weights': ['distance'],'algorithm': ['auto']}
}

In [ ]:
warnings.filterwarnings("ignore")
cross_val_performance_other = clfd.cross_validate_performance(clfd.clfs, other, data_dict, [0.99, 0.98, 0.95, 0.9, 0.8, 0.7, 0.6, 0.5], \
                                ["accuracy", "f1", "recall", "precision", "AUC_ROC"])

In [41]:
cross_val_performance_other.to_pickle("cross_val_performance_other_final")

In [42]:
ct2 = cross_val_performance_other

In [44]:
ct2[(ct2.method=="KNN") & (ct2.cross_validate_index=="tmp_label2") & (ct2.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                                KNN
accuracy                                                         0.707886
f1                                                                 0.1066
recall                                                          0.0629241
precision                                                        0.348485
AUC_ROC                                                          0.544768
threshold                                                            0.95
parameters              KNN with parameters : {'n_neighbors': 10, 'lea...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object

In [45]:
ct2[(ct2.method=="SVM") & (ct2.cross_validate_index=="tmp_label2") & (ct2.threshold==0.95)].sort_values(by="precision", ascending=False).iloc[0]

method                                                                SVM
accuracy                                                         0.702431
f1                                                              0.0899147
recall                                                          0.0530751
precision                                                        0.293939
AUC_ROC                                                          0.503502
threshold                                                            0.95
parameters              SVM with parameters : {'decision_function_shap...
cross_validate_index                                           tmp_label2
Name: 0, dtype: object